[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/langchain-ai/langchain-academy/blob/main/module-0/basics.ipynb) [![Open in LangChain Academy](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66e9eba12c7b7688aa3dbb5e_LCA-badge-green.svg)](https://academy.langchain.com/courses/take/intro-to-langgraph/lessons/56295530-getting-set-up-video-guide)

# LangChain Academy

Welcome to LangChain Academy! 

## Context

At LangChain, we aim to make it easy to build LLM applications. One type of LLM application you can build is an agent. There’s a lot of excitement around building agents because they can automate a wide range of tasks that were previously impossible. 

In practice though, it is incredibly difficult to build systems that reliably execute on these tasks. As we’ve worked with our users to put agents into production, we’ve learned that more control is often necessary. You might need an agent to always call a specific tool first or use different prompts based on its state. 

To tackle this problem, we’ve built [LangGraph](https://docs.langchain.com/oss/python/langgraph/overview) — a framework for building agent and multi-agent applications. Separate from the LangChain package, LangGraph’s core design philosophy is to help developers add better precision and control into agent workflows, suitable for the complexity of real-world systems.

## Course Structure

The course is structured as a set of modules, with each module focused on a particular theme related to LangGraph. You will see a folder for each module, which contains a series of notebooks. A video will accompany each notebook to help walk through the concepts, but the notebooks are also stand-alone, meaning that they contain explanations and can be viewed independently of the videos. Each module folder also contains a `studio` folder, which contains a set of graphs that can be loaded into [LangSmith Studio](https://docs.langchain.com/langsmith/quick-start-studio), our IDE for building LangGraph applications.

## Setup

Before you begin, please follow the instructions in the `README` to create an environment and install dependencies.

## Chat models

In this course, we'll use Chat Models, which take a sequence of messages as input and return messages as output. LangChain supports many models via [third-party integrations](https://docs.langchain.com/oss/python/integrations/chat). By default, the course will use  [ChatOpenAI](https://docs.langchain.com/oss/python/integrations/chat/openai) because it is both popular and performant. As noted, please ensure that you have an `OPENAI_API_KEY`.

Let's check that your `OPENAI_API_KEY` is set and, if not, you will be asked to enter it.

In [1]:
%%capture --no-stderr
%pip install --quiet -U langchain_openai langchain_core langchain_community langchain-tavily

In [12]:
import os
import getpass
from dotenv import load_dotenv

# 1. Load environment variables from the .env file
# This reads your .env file and loads variables into os.environ
load_dotenv()

def _set_env(var: str):
    # Check if the variable is set in the OS environment (including those loaded by .env)
    env_value = os.environ.get(var)
    if not env_value:
        # If not set, prompt the user for input
        env_value = getpass.getpass(f"{var}: ")
    
    # Set the environment variable for the current process
    os.environ[var] = env_value

# 2. Ensure AWS credentials are set
# If they are in your .env file, they will be used automatically.
# If not, you will be prompted to enter them.
_set_env("AWS_ACCESS_KEY_ID")
_set_env("AWS_SECRET_ACCESS_KEY")
_set_env("AWS_DEFAULT_REGION") # e.g., us-east-1

# Optional: If you are using temporary credentials (SSO), uncomment the line below:
# _set_env("AWS_SESSION_TOKEN") 

# 3. LangSmith Configuration (from your example)
_set_env("LANGSMITH_API_KEY")
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "langchain-academy"

print("✅ Environment setup complete.")

# --- OLD CONFIGURATION COMMENTED OUT BELOW ---
# # import os, getpass

# def _set_env(var: str):
#     if not os.environ.get(var):
#         os.environ[var] = getpass.getpass(f"{var}: ")

# # _set_env("OPENAI_API_KEY")

[Here](https://docs.langchain.com/oss/python/langchain/models) is a useful how-to for all the things that you can do with chat models, but we'll show a few highlights below. If you've run `pip install -r requirements.txt` as noted in the README, then you've installed the `langchain-openai` package. With this, we can instantiate our `ChatOpenAI` model object. You can see pricing for various models [here](https://openai.com/api/pricing/). The notebooks will default to `gpt-4o` because it offers a good balance of quality, price, and speed, but you can also opt for the lower-priced `gpt-3.5` series or more recent models.

There are [a few standard parameters](https://docs.langchain.com/oss/python/langchain/models#parameters) that we can set with chat models. Two of the most common are:

* `model`: the name of the model
* `temperature`: the sampling temperature

`Temperature` controls the randomness or creativity of the model's output where low temperature (close to 0) is more deterministic and focused outputs. This is good for tasks requiring accuracy or factual responses. High temperature (close to 1) is good for creative tasks or generating varied responses. 

In [6]:
from langchain_aws import ChatBedrockConverse

# 1. CONFIGURACIÓN PARA DEEPSEEK-R1 (Razonamiento Complejo)
llm_deepseek_r1 = ChatBedrockConverse(
    model="us.deepseek.r1-v1:0",
    region_name="us-east-1",
    temperature=0.6,
    max_tokens=8192,
    top_p=0.95,
)

# 2. CONFIGURACIÓN PARA DEEPSEEK-V3
llm_deepseek_v3 = ChatBedrockConverse(
    model="us.deepseek.v3-v1:0",
    region_name="us-east-1",
    temperature=0.7,
    max_tokens=4096,
)

# 3. CONFIGURACIÓN PARA LLAMA 4 SCOUT
llm_scout = ChatBedrockConverse(
    model="us.meta.llama4-scout-17b-instruct-v1:0",
    region_name="us-east-1",
    temperature=0.5,
    max_tokens=2048,
    top_p=0.9,
)

# 4. CONFIGURACIÓN PARA LLAMA 4 MAVERICK
llm_maverick = ChatBedrockConverse(
    model="us.meta.llama4-maverick-17b-instruct-v1:0",
    region_name="us-east-1",
    temperature=0.5,
    max_tokens=2048,
    top_p=0.9,
)

# 5. CONFIGURACIÓN PARA AMAZON NOVA LITE
llm_nova_lite = ChatBedrockConverse(
    model="amazon.nova-lite-v1:0",
    region_name="us-east-1",
    temperature=0.5,
    max_tokens=2048,
    top_p=0.9,
)

# 6. CONFIGURACIÓN PARA AMAZON NOVA MICRO
llm_nova_micro = ChatBedrockConverse(
    model="amazon.nova-micro-v1:0",
    region_name="us-east-1",
    temperature=0.5,
    max_tokens=2048,
    top_p=0.9,
)

# 7. CONFIGURACIÓN PARA AMAZON NOVA PRO
llm_nova_pro = ChatBedrockConverse(
    model="amazon.nova-pro-v1:0",
    region_name="us-east-1",
    temperature=0.5,
    max_tokens=2048,
    top_p=0.9,
)

# Seleccionar el LLM activo
llm = llm_scout

Chat models in LangChain have a number of [default methods](https://reference.langchain.com/python/langchain_core/runnables). For the most part, we'll be using:

* [stream](https://docs.langchain.com/oss/python/langchain/models#stream): stream back chunks of the response
* [invoke](https://docs.langchain.com/oss/python/langchain/models#invoke): call the chain on an input

And, as mentioned, chat models take [messages](https://docs.langchain.com/oss/python/langchain/messages) as input. Messages have a role (that describes who is saying the message) and a content property. We'll be talking a lot more about this later, but here let's just show the basics.

In [7]:
from langchain_core.messages import HumanMessage

# Create a message
msg = HumanMessage(content="Hello world", name="Lance")

# Message list
messages = [msg]

# Invoke the model with a list of messages 
llm_llama.invoke(messages)

AIMessage(content='\n\nHello! How can I assist you today?', additional_kwargs={'usage': {'prompt_tokens': 12, 'completion_tokens': 11, 'cache_read_input_tokens': 0, 'cache_write_input_tokens': 0, 'total_tokens': 23}, 'stop_reason': 'stop', 'thinking': {}, 'model_id': 'us.meta.llama4-scout-17b-instruct-v1:0'}, response_metadata={'usage': {'prompt_tokens': 12, 'completion_tokens': 11, 'cache_read_input_tokens': 0, 'cache_write_input_tokens': 0, 'total_tokens': 23}, 'stop_reason': 'stop', 'thinking': {}, 'model_id': 'us.meta.llama4-scout-17b-instruct-v1:0', 'model_provider': 'bedrock', 'model_name': 'us.meta.llama4-scout-17b-instruct-v1:0'}, id='lc_run--019b8bc7-bb4a-7a02-b5a0-b6df1a22c276-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 12, 'output_tokens': 11, 'total_tokens': 23, 'input_token_details': {'cache_creation': 0, 'cache_read': 0}})

We get an `AIMessage` response. Also, note that we can just invoke a chat model with a string. When a string is passed in as input, it is converted to a `HumanMessage` and then passed to the underlying model.


In [8]:
llm_nova.invoke("hello world")

AIMessage(content="Hello! How can I assist you today? If you have any questions or need information on a specific topic, feel free to ask. Whether it's about technology, science, entertainment, or anything else, I'm here to help. If you're just saying hi, it's great to meet you, and I hope you're having a wonderful day!", additional_kwargs={}, response_metadata={'ResponseMetadata': {'RequestId': 'df4681ec-1ab2-4296-9128-030f07341f63', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Mon, 05 Jan 2026 01:31:30 GMT', 'content-type': 'application/json', 'content-length': '503', 'connection': 'keep-alive', 'x-amzn-requestid': 'df4681ec-1ab2-4296-9128-030f07341f63'}, 'RetryAttempts': 0}, 'stopReason': 'end_turn', 'metrics': {'latencyMs': [548]}, 'model_provider': 'bedrock_converse', 'model_name': 'amazon.nova-lite-v1:0'}, id='lc_run--019b8bc7-dd37-76b0-aebc-5453b8e116c0-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 2, 'output_tokens': 77, 'total_tokens': 79, 'input

In [9]:
llm_ds.invoke("hello world")

AIMessage(content='Hello! How can I assist you today? 😊', additional_kwargs={'usage': {'prompt_tokens': 21, 'completion_tokens': 12, 'cache_read_input_tokens': 0, 'cache_write_input_tokens': 0, 'total_tokens': 33}, 'stop_reason': None, 'thinking': {}, 'model_id': 'us.deepseek.r1-v1:0'}, response_metadata={'usage': {'prompt_tokens': 21, 'completion_tokens': 12, 'cache_read_input_tokens': 0, 'cache_write_input_tokens': 0, 'total_tokens': 33}, 'stop_reason': None, 'thinking': {}, 'model_id': 'us.deepseek.r1-v1:0', 'model_provider': 'bedrock', 'model_name': 'us.deepseek.r1-v1:0'}, id='lc_run--019b8bc8-0778-7f71-9211-4dcef3d26fa8-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 21, 'output_tokens': 12, 'total_tokens': 33, 'input_token_details': {'cache_creation': 0, 'cache_read': 0}})

The interface is consistent across all chat models and models are typically initialized once at the start up each notebooks. 

So, you can easily switch between models without changing the downstream code if you have strong preference for another provider.


## Search Tools

You'll also see [Tavily](https://tavily.com/) in the README, which is a search engine optimized for LLMs and RAG, aimed at efficient, quick, and persistent search results. As mentioned, it's easy to sign up and offers a generous free tier. Some lessons (in Module 4) will use Tavily by default but, of course, other search tools can be used if you want to modify the code for yourself.

In [13]:
_set_env("TAVILY_API_KEY")

In [14]:
from langchain_tavily import TavilySearch  # updated at 1.0

tavily_search = TavilySearch(max_results=3)

data = tavily_search.invoke({"query": "What is LangGraph?"})
search_docs = data.get("results", data)

/home/juansebas7ian/langchain-academy/lc-academy-env/lib/python3.12/site-packages/langchain_tavily/tavily_research.py:97: UserWarning: Field name "output_schema" in "TavilyResearch" shadows an attribute in parent "BaseTool"
  class TavilyResearch(BaseTool):  # type: ignore[override, override]
/home/juansebas7ian/langchain-academy/lc-academy-env/lib/python3.12/site-packages/langchain_tavily/tavily_research.py:97: UserWarning: Field name "stream" in "TavilyResearch" shadows an attribute in parent "BaseTool"
  class TavilyResearch(BaseTool):  # type: ignore[override, override]


In [15]:
search_docs

[{'url': 'https://docs.langchain.com/oss/javascript/langgraph/overview',
  'title': 'LangGraph overview - Docs by LangChain',
  'content': 'Trusted by companies shaping the future of agents— including Klarna, Replit, Elastic, and more— LangGraph is a low-level orchestration framework and runtime for building, managing, and deploying long-running, stateful agents. Before using LangGraph, we recommend you familiarize yourself with some of the components used to build agents, starting with models and tools. If you are just getting started with agents or want a higher-level abstraction, we recommend you use LangChain’s agents that provide pre-built architectures for common LLM and tool-calling loops. LangGraph is focused on the underlying capabilities important for agent orchestration: durable execution, streaming, human-in-the-loop, and more. LangGraph provides low-level supporting infrastructure for *any* long-running, stateful workflow or agent. While LangGraph can be used standalone, i